<h1>Biomedical Image Analysis in Python (Part - 3)</h1>
<h3>by Sakib Reza<h3>
<h2>Segment the Heart</h2>
Here, we'll work with <b>magnetic resonance (MR)</b> imaging data from the <b>Sunnybrook Cardiac Dataset</b>. The full image is a 3D time series spanning a single heartbeat. These data are used by radiologists to measure the ejection fraction: the proportion of blood ejected from the left ventricle during each stroke.

To begin, segment the left ventricle from a single slice of the volume (im). First, you'll filter and mask the image; then you'll label each object with ndi.label().
<img src="img14.jpg">

In [ ]:
import imageio
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

# Smooth intensity values
im_filt = ndi.median_filter(im, size=3)

# Select high-intensity pixels
mask_start = np.where(im_filt>60, 1, 0)
mask = ndi.binary_closing(mask_start)

# Label the objects in "mask"
labels, nlabels = ndi.label(mask)
print('Num. Labels:',nlabels)

# Create a `labels` overlay
overlay = np.where(labels>0, labels, np.nan)

# Use imshow to plot the overlay
plt.imshow(overlay, cmap='rainbow', alpha=0.75)
format_and_render_plot()

<b>Output:</b><br>
Num. Labels: 26
<img src="img15.jpg">

<h2>Select objects</h2>
Labels are like object "handles" - they give you a way to pick up whole sets of pixels at a time. To select a particular object:

1. Find the label value associated with the object.<br>
2. Create a mask of matching pixels.<br>

Here we will create a labeled array from the provided mask. Then, find the label value for the centrally-located left ventricle, and create a mask for it.

<img src="img16.jpg">

In [ ]:
# Label the image "mask"
labels, nlabels = ndi.label(mask)

# Select left ventricle pixels
lv_val = labels[128, 128]
lv_mask = np.where(labels == lv_val, 1, np.nan)

# Overlay selected label
plt.imshow(lv_mask, cmap='rainbow')
plt.show()

<b>Output:</b>
<img src="img17.jpg">

<h2>Extract objects</h2>
Extracting objects from the original image eliminates unrelated pixels and provides new images that can be analyzed independently.

The key is to crop images so that they only include the object of interest. The range of pixel indices that encompass the object is the bounding box.

Here, We will use ndi.find_objects() to create a new image containing only the left ventricle.

In [ ]:
# Create left ventricle mask
labels, nlabels = ndi.label(mask)
lv_val = labels[128, 128]
lv_mask = np.where(labels == lv_val, 1, 0)

# Find bounding box of left ventricle
bboxes = ndi.find_objects(lv_mask)
print('Number of objects:', len(bboxes))
print('Indices for first box:', bboxes[0])

# Crop to the left ventricle (index 0)
im_lv = im[bboxes[0]]

# Plot the cropped image
plt.imshow(im_lv)
format_and_render_plot()

<b>Output:</b><br>
Number of objects: 1<br>
Indices for first box: (slice(107, 149, None), slice(116, 162, None))<br>
<img src="img17.jpg">


<h2>Measure variance</h2>
SciPy measurement functions allow you to tailor measurements to specific sets of pixels:

Specifying 'labels' restricts the mask to non-zero pixels.
Specifying 'index' value(s) returns a measure for each label value.
For this exercise, calculate the intensity variance of vol with respect to different pixel sets. We have provided the 3D segmented image as labels: label 1 is the left ventricle and label 2 is a circular sample of tissue.
<img src="gg.gif">

In [ ]:
# Variance for all pixels
var_all = ndi.variance(vol)
print('All pixels:', var_all)

# Variance for labeled pixels
var_labels = ndi.variance(vol, labels)
print('Labeled pixels:', var_labels)

# Variance for each object
var_objects = ndi.variance(vol, labels, index=[1,2])
print('Left ventricle:', var_objects[0])
print('Other tissue:', var_objects[1])

<b>Output:</b><br>
All pixels: 840.4457526156154<br>
Labeled pixels: 2166.5887761076724<br>
Left ventricle: 1123.4641972021984<br>
Other tissue: 1972.7151849347783<br>

<h2>Separate histograms</h2>
A poor tissue segmentation includes multiple tissue types, leading to a wide distribution of intensity values and more variance.

On the other hand, a perfectly segmented left ventricle would contain only blood-related pixels, so the histogram of the segmented values should be roughly bell-shaped.

Here, we will compare the intensity distributions within vol for the listed sets of pixels. Use ndi.histogram, which also accepts labels and index arguments.

In [ ]:
# Create histograms for selected pixels
hist1 = ndi.histogram(vol, min=0, max=255, bins=256)
hist2 = ndi.histogram(vol, 0, 255, 256, labels=labels)
hist3 = ndi.histogram(vol, 0, 255, 256, labels=labels, index=1)

# Plot the histogram density
plt.plot(hist1 / hist1.sum(), label='All pixels')
plt.plot(hist2 / hist2.sum(), label='All labeled pixels')
plt.plot(hist3 / hist3.sum(), label='Left ventricle')
format_and_render_plot()

<b>Output:</b><br>
<img src="img19.jpg">


<h2>Calculate distance</h2>
A distance transformation calculates the distance from each pixel to a given point, usually the nearest background pixel. This allows you to determine which points in the object are more interior and which are closer to edges.

Here, we will use the Euclidian distance transform on the left ventricle object in labels.

In [ ]:
# Calculate left ventricle distances
lv = np.where(labels == 1, 1, 0)
dists = ndi.distance_transform_edt(lv, sampling = vol.meta['sampling'])

# Report on distances
print('Max distance (mm):', ndi.maximum(dists))
print('Max location:', ndi.maximum_position(dists))

# Plot overlay of distances
overlay = np.where(dists[5] > 0, dists[5], np.nan) 
plt.imshow(overlay, cmap='hot')
format_and_render_plot()

<b>Output:</b><br>
Max distance (mm): 16.320510697696196<br>
Max location: (5, 129, 137)<br>

<img src="img20.jpg">


<h2>Pinpoint center of mass</h2>
The distance transformation reveals the most embedded portions of an object. On the other hand, ndi.center_of_mass() returns the coordinates for the center of an object.

The "mass" corresponds to intensity values, with higher values pulling the center closer to it.

Here,we will calculate the center of mass for the two labeled areas. Then, plot them on top of the image.

In [ ]:
# Extract centers of mass for objects 1 and 2
coms = ndi.center_of_mass(vol,
                       labels, 
                       index=[1,2])
print('Label 1 center:', coms[0])
print('Label 2 center:', coms[1])

# Add marks to plot
for c0, c1, c2 in coms:
    plt.scatter(c2, c1, s=100, marker='o')
plt.show()

<b>Output:</b><br>
Label 1 center: (4.9149927898701, 125.72786150646698, 141.42957762070142)<br>
Label 2 center: (5.458351990999034, 121.50943176720855, 121.72954667630684)<br>

<img src="img21.jpg">


<h2>Summarize the time series</h2>
The ejection fraction is the proportion of blood squeezed out of the left ventricle each heartbeat. To calculate it, radiologists have to identify the maximum volume (systolic volume) and the minimum volume (diastolic volume) of the ventricle.

Here we will create a time series of volume calculations. There are 20 time points in both vol_ts and labels. The data is ordered by (time, plane, row, col).

<img src="gg1.gif">

In [ ]:
# Create an empty time series
ts = np.zeros(20)

# Calculate volume at each voxel
d0, d1, d2, d3 = vol_ts.meta['sampling']
dvoxel = d1*d2*d3

# Loop over the labeled arrays
for t in range(20):
    nvoxels = ndi.sum(1, labels[t], index=1)
    ts[t] = nvoxels*dvoxel

# Plot the data
plt.plot(ts)
format_and_render_plot()

<b>Output:</b><br>
<img src="img22.jpg">

<h2>Measure ejection fraction</h2>
The ejection fraction is defined as:

(Vmax−Vmin)/Vmax

...where V is left ventricle volume for one 3D timepoint.

To close our investigation, plot slices from the maximum and minimum volumes by analyzing the volume time series (ts). we will calculate the ejection fraction.

In [ ]:
# Get index of max and min volumes
tmax = np.argmax(ts)
tmin = np.argmin(ts)

# Plot the largest and smallest volumes
fig, axes = plt.subplots(2,1)
axes[0].imshow(vol_ts[tmax, 4], vmax=160)
axes[1].imshow(vol_ts[tmin, 4], vmax=160)
format_and_render_plots()

# Calculate ejection fraction
ej_vol = ts.max() - ts.min()
ej_frac = ej_vol / ts.max()
print('Est. ejection volume (mm^3):', ej_vol)
print('Est. ejection fraction:', ej_frac)

<b>Output:</b><br>
<img src="img23.jpg">
Est. ejection volume (mm^3): 31268.00536236538<br>
Est. ejection fraction: 0.3202054794520548<br>